# Python и интернет. Модуль requests

**План**:

1. Запросы
2. Requests
4. bs4
5. Задание на семинар

## Как выкачать интернет
Современный Интернет предоставляет лингвистам большое количество языковых данных: электронные газеты и журналы, блоги, форумы, социальные сети и т.д. Например, можно найти в сети много-много текстов и собрать корпус, или найти все газетные статьи и блог-посты про какую-нибудь корпорацию и проанализировать тональность сообщений. Сейчас мы научимся заниматься выкачиванием страниц из интернета с помощью Python.

Для скачивания HTML-страниц в питоне есть несколько библиотек **urllib** и **requests**

## Requests

Можно почитать еще [тут](https://realpython.com/python-requests/)


Допустим, мы хотим скачать главную страницу Хабрахабра.

На самом деле, когда мы хотим открыть какую-то страницу в интернете, наш браузер отправляет на сервер **запрос** ("Привет, сервер! я хочу код страницы по вот такому адресу!"), а сервер затем отправляет ответ ("Привет! Вот код страницы: ...").
Чтобы получить страницу через питон, нужно сформировать **запрос** на сервер так же, как это делает браузер:

In [1]:
import requests

In [2]:
response = requests.get("https://habr.com/ru/")

В response теперь лежит отет сервера. Это не прост html-код страницы, а еще дополнительная информация. Если мы просто выведем этот response, нам покажется только код (200 - все ок).

In [3]:
response

<Response [200]>

In [4]:
response.status_code

200

А вот в атрибуте text уже лежит html-код

In [5]:
print(response.text[:210])

<!DOCTYPE html>
<html lang="ru" data-vue-meta="%7B%22lang%22:%7B%22ssr%22:%22ru%22%7D%7D">
<head >
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width,initial-scale=1.0,viewport-fit=cov


Иногда сайт блокирует запросы, если их посылает не настоящий браузер с пользователем, а какой-то бот (например, так делает Гугл или Википедия). Иногда сайты присылают разные версии страниц, разным браузерам.  
По этим причинам полезно бывает писать скрипт, который умеет притворяться то одним, то другим браузером.
Когда мы пытаемся получить страницу в питоне, наш код по умолчанию честно сообщает серверу, что он является программой на питоне. Он говорит что-то вроде "Привет, я Python-urllib/3.5". 
Но можно, например, представиться Мозиллой:

In [6]:
url = 'https://habr.com/ru/'  # адрес страницы, которую мы хотим скачать
user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'  # хотим притворяться браузером

response = requests.get("https://habr.com/ru/", headers={'User-Agent':user_agent})

Или использовать специальную библиотеку

In [7]:
from fake_useragent import UserAgent

In [8]:
user_agent = UserAgent().chrome
user_agent

'Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1464.0 Safari/537.36'

In [9]:
response = requests.get("https://habr.com/ru/", headers={'User-Agent':user_agent})

Можно посмотреть, что еще можно достать из response.

Функция ```dir``` выдает список методов и параметров объекта.

In [10]:
[i for i in dir(response) if not i.startswith("_")]

['apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

Кодировка

In [11]:
response.encoding

'utf-8'

Заголовки (техническая информация)

In [12]:
dict(response.headers)

{'Server': 'QRATOR',
 'Date': 'Thu, 11 Nov 2021 15:08:19 GMT',
 'Content-Type': 'text/html; charset=utf-8',
 'Transfer-Encoding': 'chunked',
 'Connection': 'keep-alive',
 'Keep-Alive': 'timeout=15',
 'Vary': 'Accept-Encoding, Accept-Encoding',
 'X-DNS-Prefetch-Control': 'off',
 'X-Frame-Options': 'SAMEORIGIN',
 'Strict-Transport-Security': 'max-age=15552000; includeSubDomains, max-age=31536000; includeSubDomains',
 'X-Download-Options': 'noopen',
 'X-Content-Type-Options': 'nosniff',
 'X-XSS-Protection': '1; mode=block',
 'ETag': 'W/"27c34-rbO7zG5CbGe7CAV9OdNCuny+sdg"',
 'X-Proxy-Cache-Status': 'HIT',
 'X-Request-Id': 'c7a483e4f1c8c797df82a2de1fa725dd',
 'Content-Encoding': 'gzip'}

Адрес запроса

In [13]:
response.url

'https://habr.com/ru/all/'

In [14]:
print(response.text[:300])

<!DOCTYPE html>
<html lang="ru" data-vue-meta="%7B%22lang%22:%7B%22ssr%22:%22ru%22%7D%7D">
<head >
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width,initial-scale=1.0,viewport-fit=cover">
  <title>Все публикации подряд / Хабр</title>
  <style>
    /* cyrillic-ext */
    @f


Ура, всё на месте!

Но что всё это значит? Что такое html и как вообще из него доставать какую-то информацию?

Ответ: по **тегам**! Например, в куске html сверху есть теги `<title> </title>` (теги всегда обрамляют с двух сторон то, что находится под этим тегом). В `<title>` в данном случае лежит заголовок этой интернет-страницы.

Существует несколько вариантов, как достать что-то из определенного тега, например, достать заголовок:

1. регулярные выражения (плохой вариант)
2. специальные библиотеки питона, например, BeautifulSoup (bs4) или lxml (хороший вариант)

## BeautifulSoup

Код страницы парсится как иерархия тегов (как они есть в html коде, один вложен в другой) и можно искать элементы с помощью разных методов, использовать атрибуты и т.д.

In [15]:
from bs4 import BeautifulSoup

Сначала инициализируем объект BeautifulSoup. Потом применим метод find и в скобочках укажем теги, по которым ищем. У некоторых тегов в html (как и в нашем случае) бывает еще и class и какие-нибудь еще атрибуты. Такие вещи мы задаем словариком.

Этот запрос вернёт нам только первый заголовок. То есть первое вхождение такого тега в нашем html файле.

In [16]:
soup = BeautifulSoup(response.text, 'html.parser')  # инициализируем (создаем) soup

post = soup.find('h2', {'class': 'tm-article-snippet__title_h2'})
print(post.get_text(), end="\n\n\n")
print(post.prettify())

Ajax-запросы нативными средствами Joomla


<h2 class="tm-article-snippet__title tm-article-snippet__title_h2">
 <a class="tm-article-snippet__title-link" data-article-link="" href="/ru/post/588651/">
  <span>
   Ajax-запросы нативными средствами Joomla
  </span>
 </a>
</h2>



Но мы хотим получить все заголовки постов! Метод find_all возвращает массив всех элементов с тегом указанным в скобках. По нему можно итерироваться.

In [17]:
for post in soup.find_all('h2', {'class': 'tm-article-snippet__title_h2'})[:3]:
    print(post.get_text())
    print(post.prettify())

    print('-- '*10)  # для красоты

Ajax-запросы нативными средствами Joomla
<h2 class="tm-article-snippet__title tm-article-snippet__title_h2">
 <a class="tm-article-snippet__title-link" data-article-link="" href="/ru/post/588651/">
  <span>
   Ajax-запросы нативными средствами Joomla
  </span>
 </a>
</h2>

-- -- -- -- -- -- -- -- -- -- 
Prisma: Полноценный ORM для Node.js и TypeScript
<h2 class="tm-article-snippet__title tm-article-snippet__title_h2">
 <a class="tm-article-snippet__title-link" data-article-link="" href="/ru/post/588562/">
  <span>
   Prisma: Полноценный ORM для Node.js и TypeScript
  </span>
 </a>
</h2>

-- -- -- -- -- -- -- -- -- -- 
Визуализация данных с помощью веб-фреймворка Dash
<h2 class="tm-article-snippet__title tm-article-snippet__title_h2">
 <a class="tm-article-snippet__title-link" data-article-link="" href="/ru/post/588645/">
  <span>
   Визуализация данных с помощью веб-фреймворка Dash
  </span>
 </a>
</h2>

-- -- -- -- -- -- -- -- -- -- 


## Задание на семинар 1

А что если мы хотим зайти еще глубже по дереву тегов и, например, для каждого заголовка поста найти никнейм юзера, который написал этот пост, и время написания поста?

Для этого надо снова зайти в просмотор кода страницы и увидеть, что там просиходит что-то такое:

(Заодно обратите внимание, как пишутся комменты в html)

In [ ]:
<article id="588645" data-navigatable="" tabindex="0" class="tm-articles-list__item">
   <div class="tm-article-snippet">
      <div class="tm-article-snippet__meta-container">
         <div class="tm-article-snippet__meta">
            <span class="tm-user-info tm-article-snippet__author">
               <a href="/ru/users/NewTechAudit/" title="NewTechAudit" class="tm-user-info__userpic">
                  <div class="tm-entity-image">
                     <img alt="" height="24" loading="lazy" src="//habrastorage.org/r/w32/getpro/habr/avatars/62e/cf1/3f1/62ecf13f1b5e83c0cced637ff786d5a1.png" width="24" class="tm-entity-image__pic">
                  </div>
               </a>
               <span class="tm-user-info__user">
               <a href="/ru/users/NewTechAudit/" class="tm-user-info__username">NewTechAudit</a>
               </span>
            </span>
            <span class="tm-article-snippet__datetime-published">
            <time datetime="2021-11-11T14:36:40.000Z" title="2021-11-11, 17:36">сегодня в 17:36</time>
            </span>
         </div>
         <!---->
      </div>
      <h2 class="tm-article-snippet__title tm-article-snippet__title_h2">
         <a href="/ru/post/588645/" data-article-link="" class="tm-article-snippet__title-link">
         <span>Визуализация данных с помощью веб-фреймворка Dash</span>
         </a>
      </h2>
      <div class="tm-article-snippet__hubs">
         <span class="tm-article-snippet__hubs-item">
         <a href="/ru/hub/programming/" class="tm-article-snippet__hubs-item-link">
         <span>Программирование</span>
         <span title="Профильный хаб" class="tm-article-snippet__profiled-hub">*</span>
         </a>
         </span>
         <span class="tm-article-snippet__hubs-item">
         <a href="/ru/hub/python/" class="tm-article-snippet__hubs-item-link">
         <span>Python</span> 
         <span title="Профильный хаб" class="tm-article-snippet__profiled-hub">*</span>
         </a></span>
         <span class="tm-article-snippet__hubs-item">
            <a href="/ru/hub/data_visualization/" class="tm-article-snippet__hubs-item-link">
               <span>Визуализация данных</span> 
               <!---->
            </a>
         </span>
      </div>
      <!---->
      <!---->
      <div class="tm-article-body tm-article-snippet__lead">
         <div class="tm-article-snippet__cover tm-article-snippet__cover_cover">
            <img src="https://habrastorage.org/getpro/habr/upload_files/1a0/17f/c29/1a017fc2904dc1ac1811edcc30648447.png" class="tm-article-snippet__lead-image" style="object-position:0% 0%;">
         </div>
         <div class="article-formatted-body article-formatted-body_version-2">
            <p>Dash является довольно интересным Веб-фреймворком для визуализации данных и имеет в семе довольно много полезных функций в сочетании с простотой их применения. </p>
         </div>
         <a href="/ru/post/588645/" class="tm-article-snippet__readmore">
         <span>Читать далее</span>
         </a>
      </div>
   </div>
   <div class="tm-data-icons">
      <!---->
      <div class="tm-votes-meter tm-data-icons__item">
         <svg height="16" width="16" class="tm-svg-img tm-votes-meter__icon tm-votes-meter__icon_small">
            <title>Рейтинг</title>
            <use xlink:href="/img/megazord-v24.ce74655c.svg#counter-rating"></use>
         </svg>
         <span class="tm-votes-meter__value tm-votes-meter__value_small">0</span>
      </div>
      <span title="Количество просмотров" class="tm-icon-counter tm-data-icons__item">
         <svg height="16" width="16" class="tm-svg-img tm-icon-counter__icon">
            <title>Просмотры</title>
            <use xlink:href="/img/megazord-v24.ce74655c.svg#counter-views"></use>
         </svg>
         <span class="tm-icon-counter__value">96</span>
      </span>
      <button title="Добавить в закладки" type="button" class="bookmarks-button tm-data-icons__item">
         <span title="Добавить в закладки" class="tm-svg-icon__wrapper bookmarks-button__icon">
            <svg height="16" width="16" class="tm-svg-img tm-svg-icon">
               <title>Добавить в закладки</title>
               <use xlink:href="/img/megazord-v24.ce74655c.svg#counter-favorite"></use>
            </svg>
         </span>
         <span title="Количество пользователей, добавивших публикацию в закладки" class="bookmarks-button__counter">0</span>
      </button>
      <div title="Читать комментарии" class="tm-article-comments-counter-link tm-data-icons__item">
         <a href="/ru/post/588645/comments/" class="tm-article-comments-counter-link__link">
            <svg height="16" width="16" class="tm-svg-img tm-article-comments-counter-link__icon">
               <title>Комментарии</title>
               <use xlink:href="/img/megazord-v24.ce74655c.svg#counter-comments"></use>
            </svg>
            <span class="tm-article-comments-counter-link__value">
            0
            </span>
         </a>
         <!---->
      </div>
      <!----> 
      <div class="v-portal" style="display:none;"></div>
   </div>
</article>

## Задание на семинар 2

Скачать главную страницу Яндекс.Погоды и
    
- а) распечатать сегодняшнюю температуру и облачность
- б) распечатать время восхода и заката
- в) погоду на завтра

## Хорошая статья про это все

[https://sysblok.ru/courses/obkachka-sajtov-svoimi-rukami-razbiraemsja-s-html/](https://sysblok.ru/courses/obkachka-sajtov-svoimi-rukami-razbiraemsja-s-html/)